### Plain Programm
Here follows a version of the program without any instructions. 

In [ ]:
using LinearAlgebra, GaussQuadrature

struct QuadElement4{I}
    id::I
    Nodes::Vector{I}
end

struct PlaneModel
    NodeDict::Dict
    ElemDict::Dict
    NSetDict::Dict
    ESetDict::Dict
    DOFDict::Dict
end

struct CellValues{T, I}
    𝐍::Vector{T}
    𝐁::Matrix{T}
    𝐉::Matrix{T}
    𝐁ε::Matrix{T}
    Coords::Matrix{T}
    ξVec::Vector{T}
    wVec::Vector{T}
    celldofs::Vector{I}
end

function legendre2d(nq)
    ξTemp, wTemp = legendre(nq)
    
    ξMat = [(0.0,0.0) for i∈1:nq, j∈1:nq]
    wMat = zeros(nq,nq)

    for i in 1:nq
        for j in 1:nq
            ξMat[i,j] = (ξTemp[i], ξTemp[j])
            wMat[i,j] = wTemp[i] * wTemp[j]
        end
    end
    
    return ξMat[:], wMat[:]
end

function CellValues(;nq=2)
    𝐍 = zeros(4)
    𝐁 = zeros(2,4)
    𝐉 = zeros(2,2)
    𝐁ε = zeros(3,8)
    Coords = zeros(4,2)
    ξVec, wVec = compute_gquad(nq)
    celldofs = zeros(Int,8)
    return CellValues(𝐍, 𝐁, 𝐉, 𝐁ε, Coords, ξVec, wVec, celldofs)
end

function celldofs!(cell_v::CellValues{T, I} ,elem::PE, DOFDict::Dict) where {TE, T, I}
    for (i, inodeid)  ∈ enumerate(elem.Nodes)
        cell_v.celldofs[i] = DOFDict[inodeid][1]
    end
end 
function getxcoord!(cell_v::CellValues{T, I} ,elem::PE, NodeDict::Dict) where {TE, T, I}
    for (i, inodeid)  ∈ enumerate(elem.Nodes)
        cell_v.Coords[i,1] = NodeDict[inodeid][1]
        cell_v.Coords[i,2] = NodeDict[inodeid][2]
    end
end

"""
    shapeFunction!(𝐍::Vector{T}, 𝐁::Vector{T}, ξ::T, η::T) where T

calculate the shape functions and their derivatives at a given point
    ^ η
    |
4 ---- 3
|      | -- > ξ
1 ---- 2

"""
function shapeFunction!(𝐍::Vector{T}, 𝐁::Vector{T}, ξ::T, η::T) where T
    # shape function and derivatives for L2 elements

    Nξp = T(1) + ξ
    Nξn = T(1) - ξ
    Nηp = T(1) + η
    Nηn = T(1) - η

    𝐍[1] = Nξn * Nηn  
    𝐍[2] = Nξp * Nηn 
    𝐍[3] = Nξp * Nηp 
    𝐍[4] = Nξn * Nηp 
    𝐍 ./= 4

    𝐁[1,1] = - Nηn 
    𝐁[2,1] = - Nξn
    𝐁[1,2] =   Nηn
    𝐁[2,2] = - Nξp
    𝐁[1,3] =   Nηp
    𝐁[2,3] =   Nξp
    𝐁[1,4] = - Nηp
    𝐁[2,4] =   Nξn
    𝐁 ./= 4

    return nothing
end 

function reinit!(cell_v::CellValues{T}, elem::QuadElement4{I}, ξ::T, η::T) where {I, T, TE}

    shapeFunction!(cell_v.𝐍, cell_v.𝐁, elem, ξ, η)
    nnode = 4
    ndim = 2
    
    cell_v.𝐉 .= cell_v.𝐁 * cell_v.Coords
    cell_v.𝐁 .= inv(cell_v.𝐉) * cell_v.𝐁
    J = det(ell_v.𝐉)
    
    fill!(cell_v.𝐁ε, 0.0)
    for inode in 1:nnode
        cell_v.𝐁ε[1,2*inode-1] = cell_v.𝐁[1,inode]
        cell_v.𝐁ε[2,2*inode]   = cell_v.𝐁[2,inode]
        cell_v.𝐁ε[3,2*inode-1] = cell_v.𝐁[2,inode]
        cell_v.𝐁ε[3,2*inode]   = cell_v.𝐁[1,inode]
    end
    
    return J
end

function compute_Kf_element!(𝐊ᵉ::Matrix{T}, 𝐟ᵉ::Vector{T}, cell_v::CellValues{T}, elem::E, mat::MI, b::T) where {T, E, MI}

    nbasefunc = length(elem.Nodes)
    n_qpoint = length(cell_v.ξVec)

    fill!(𝐊ᵉ, 0.0)
    fill!(𝐟ᵉ, 0.0)

    for q_point ∈ 1:n_qpoint
        ξ, η = cell_v.ξVec[q_point]
        w = cell_v.wVec[q_point]
        J = reinit!(cell_v, elem, ξ, η)
        dΩ = J * w

        𝐂e = compute_C(mat, ε)
        𝐊ᵉ .+= cell_v.𝐁ε' * 𝐂e * cell_v.𝐁ε * mat.t * dΩ
        𝐟ᵉ  .+= cell_v.𝐍 * b * dΩ
    end
    return nothing
end

function Assemble_Kf(PModel::PlaneModel, cell_v::CellValues, mat::MatI, b::T) where {T, MatI}

    totaldof = ndofs(BarModel.DOFDict)
    
    𝐊 = SparseMatrixCOO();
    𝐟  = zeros(totaldof)

    nbasefunc = length(cell_v.𝐍)

    𝐊ᵉ = zeros(nbasefunc, nbasefunc)
    𝐟ᵉ = zeros(nbasefunc)

    for (kelem, elem) in BarModel.ElemDict
        celldofs!(cell_v, elem, BarModel.DOFDict)
        getxcoord!(cell_v,elem, BarModel.NodeDict)

        compute_Kf_element!(𝐊ᵉ, 𝐟ᵉ, cell_v, elem, mat, b)

        add!(𝐊,cell_v.celldofs,cell_v.celldofs,𝐊ᵉ)
        𝐟[cell_v.celldofs] .+= 𝐟ᵉ
    end

    𝐊 = SparseMatrixCSC(𝐊) 

    return 𝐊, 𝐟
end

### Example of a colored grid
Creates a simple 2D grid and colors it. Save the example grid to a VTK file to show the coloring. No cells with the same color has any shared nodes (dofs). This means that it is safe to assemble in parallel as long as we only assemble one color at a time.

For this structured grid the greedy algorithm uses fewer colors, but both algorithms result in colors that contain roughly the same number of elements. For unstructured grids the greedy algorithm can result in colors with very few element. For those cases the workstream algorithm is better since it tries to balance the colors evenly.
<p align="center">
<img src="https://ferrite-fem.github.io/Ferrite.jl/stable/examples/coloring.png" width="800">
</p>
